# 知识工程-问答系统设计与实现大作业
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 数据准备

非常好的助教已经帮我们down好了数据，注意到有四个文件
- passages_multi_sentences.json (文档库) 
- train.json (问答语料) 
- train_questions.txt (问题分类训练语料) 
- test_questions.txt (问题分类测试语料) 

我们自己查看一下数据格式

![alt text](image.png)

![alt text](image-1.png)


| 文件名 (File Name)             | 来源数据集 (Source Dataset)                                  | 数据集描述 (Dataset Description)                                                                                                                               | 文件格式 (File Format)                                                                                                                                  |
| :----------------------------- | :----------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `passages_multi_sentences.json` | 百度 DuReader V2.0 (筛选处理后)                     | 包含用于检索和答案抽取的文档库，这些文档被分割成了多个句子 。                                                                                             | 实际上是JSONL，每一行是JSON 格式。每一个json有pid字段和document 字段，document是句子字符串的列表，每个字符串逻辑上是一个Sentence。                                                                                                                       |
| `train.json`                   | 百度 DuReader V2.0 (筛选处理后)                     | 问答语料，包含问题、对应的答案以及相关的文档 `pid` 。用于训练检索评估、候选答案句排序和答案抽取模块。                                                    | 也是JSONL，每一行的对象是question、pid、          answer_sentence、   answer、qid组成的，其中answer_sentence是句子的列表，但是一般是一个句子，answer和question是单个字符串，pid和qid是int                                                          |
| `train_questions.txt`          | 哈工大信息检索研究室问答系统问题集                      | 用于训练问题分类器的问题及对应的类别标签 。类别标签形如 `HUM_PERSON`，下划线前为粗分类标签 。                                                   | 文本文件 (.txt)。每行可能是一个标签和一个问题，由制表符或空格分隔。标签看起来是这个问题的类型，比如DES_DEFINITION是要求别人描述定义的问题。                                                                                                       |
| `test_questions.txt`           | 哈工大信息检索研究室问答系统问题集                      | 用于测试问题分类器性能的问题及对应的类别标签 。                                                                    | 与`train_questions.txt`一样。                                                                                                     |


主要来自两个数据集 ：

- DuReader V2.0：这是百度在2017年发布的一个阅读理解数据集 。本次实验使用的 passages_multi_sentences.json 和 train.json 就是从这个数据集中筛选和处理得到的 。
- 哈工大信息检索研究室问答系统问题集：这是一个由哈工大信息检索研究室在2004年公开的数据集 。实验中的问题分类语料（即 train_questions.txt 和 test_questions.txt）来源于此 。

## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image-2.png)


很好，这次代码格式化没有报错。

Pylance 似乎也没有明显问题。

## 实验环境准备

采用上次的作业专属环境，为了跑通最新方法，使用3.12 和 torch 2.7

```bash
conda create -n ai python=3.12
conda activate ai
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

本次作业似乎没有新的依赖，只是用到了 torch


## 原理回顾和课件复习



在作业9. 中我们进行了简单的复习，这里不再赘述。

## 1. 建立文档检索系统

首先我们来理解这个子任务的要求，
我注意到，train.json中的样本的pid，其实就是passages中的pid，是一一对应的。
我们要构建一个函数，输入train.json中的question，中间经过关键词提取, 系统读取passages_multi_sentences.json中的文档库，返回与问题相关的文档的pid列表， 使用Recall指标来评估检索系统的性能。

这里没有让我们用深度学习，而是传统的检索。

首先需要去除停用词，停用词是指在文本中出现频率非常高，但对文本意义贡献不大的词语（如“的”、“是”、“在”等）。去除它们可以减少噪音，降低计算量。注意到助教已经提供了停用词列表 `stopwords.txt`，我们可以直接使用。



In [ ]:
def load_stopwords(filepath):
    """Loads stopwords from a file into a set."""
    stopwords_set = set()
    if not os.path.exists(filepath):
        print(f"Warning: Stopwords file not found at {filepath}. Proceeding without stopwords.")
        return stopwords_set
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            stopwords_set.add(line.strip())
    print(f"Loaded {len(stopwords_set)} stopwords.")
    return stopwords_set

在助教的作业要求中，建议我们使用Whoosh https://whoosh.readthedocs.io/en/latest/intro.html

Whoosh，像其他搜索引擎库一样，通常在构建索引时内部处理文本分析（分词、去除停用词、小写化、词干提取等）。这是通过其 Schema 中的 FieldType（尤其是 TEXT 类型字段）及其关联的 Analyzer 来完成的。

作业要求中助教让我们实现的 Jieb分词和停用词去除逻辑，实际上正是 Whoosh Analyzer 需要做的事情。所以更理想和“Whoosh原生”的做法是，我们不需要显示地写一个 preprocess_data.py 把输出（词列表）直接保存成一个中间文件让 Whoosh 去读取。而是，我们将原始文本（passages_multi_sentences.json 中的句子）直接喂给 Whoosh，然后配置 Whoosh 使用一个包含 Jieba 分词和我们停用词表的自定义 Analyzer。这样，索引的建立和文本分析的逻辑就都封装在 Whoosh 内部了，更加清晰和高效。

查阅 https://whoosh.readthedocs.io/en/latest/analysis.html

接口需要我们对str进行yield Token对象。

In [ ]:
# --- Step a) Part 1: Preprocessing logic for Whoosh Analyzer and Questions ---
class ChineseTokenizer(Tokenizer):
    """Custom Whoosh Tokenizer for Chinese text using Jieba."""
    def __call__(self, text:str, **kargs):
        words = jieba.lcut(text) 
        token_instance = Token() # Create one Token object to reuse
        current_pos = 0
        for word in words:
            word_clean = word.strip()
            # Ensure it's not a stopword and not just whitespace
            if word_clean and word_clean.lower() not in STOPWORDS and not word_clean.isspace():
                token_instance.text = word_clean
                token_instance.original = word_clean
                token_instance.pos = current_pos # Assign current position
                # token_instance.startchar = start_char # Optional: if you track character offsets
                # token_instance.endchar = end_char   # Optional
                yield token_instance
                current_pos += 1

Whoosh Schema 我们需要定义一个 Schema 来描述文档的结构。至少会有一个字段用于存储文档ID (pid)，以及一个 TEXT 字段用于存储文档内容，这个内容字段将使用我们自定义的 Analyzer。

In [ ]:
schema = Schema(
        pid=ID(stored=True, unique=True),
        content=TEXT(analyzer=chinese_analyzer(), stored=False) # Content not stored to save space
    )

现在对于每条记录，提取 pid 和 document (句子列表)。将句子列表合并为单个字符串。
使用 IndexWriter.add_document() 将 pid 和合并后的文档内容添加到索引中。

In [ ]:
full_document_text = " ".join(document_sentences)

                    writer.add_document(pid=str(pid), content=full_document_text)

writer.commit()

我们只写了关键核心代码，其他读取json等细节问题就没写了。关键是知道Whoosh就是用writer来写入索引，检索的时候不区分文档的句子，只是检索文档。

然后是推理的时候，给定一个问题，要怎么查询pid。

In [ ]:
def search_documents(ix, question_text, top_n=3):
    """
    Searches the index for a given question and returns top_n pids.
    (Addresses part of step c)
    """
    # Preprocess the question to get keywords (also part of step c)
    query_keywords = preprocess_question(question_text)
    if not query_keywords:
        return []

    # Form a query string (Whoosh OR query by default for space-separated terms)
    # Using OR logic, as suggested by "命中一个关键词加一" then sum scores.
    # Whoosh's default OR for space-separated terms is a good starting point.
    query_str = " ".join(query_keywords)
    
    retrieved_pids = []
    with ix.searcher() as searcher:
        # Using MultifieldParser to search in the 'content' field.
        # You could use QueryParser for a single field too.
        # Default operator is OR. To make it AND, use qparser.AND
        query_parser = QueryParser("content", schema=ix.schema)
        query = query_parser.parse(query_str)
        
        results = searcher.search(query, limit=top_n)
        for hit in results:
            retrieved_pids.append(hit['pid']) # pid is stored as string in index
            
    return retrieved_pids

这一步关键是要用 QueryParser 和 searcher

现在我们运行

python src/retrieval_system.py

![alt text](image-3.png)

![alt text](image-4.png)

![alt text](image-5.png)

可以看到这个recall还是比较低的，这是因为我们刚才没有认真写preprocess_question，刚才就是分词+去掉停用词，刚才调代码是为了跑通Whoosh先。
现在我们来"对问题进行预处理，去除停用词，并选择合适的方法（如词性标注、依存句法以及语义角色标注等）抽取问题中的关键词"

对于Whoosh的Query Formulation，recall比较低的时候，我们使用OR查询是对的，只要有一个关键词匹配就可以了，如果precision低才适合AND查询。

对于算法，Whoosh默认使用的是 BM25F ，看起来已经考虑的够多了这个算法。




In [ ]:
import jieba.posseg as pseg # Import pseg for POS tagging

def preprocess_question(question_text, pos_tags_to_keep=None):
    """
    Preprocesses a question string using POS tagging to extract keywords.
    Args:
        question_text (str): The input question.
        pos_tags_to_keep (set, optional): A set of POS tags to consider for keywords. 
                                         Defaults to nouns, proper nouns, and verbs.
    Returns:
        list: A list of deduplicated keywords.
    """
    if pos_tags_to_keep is None:
        pos_tags_to_keep = {'n', 'nr', 'ns', 'nt', 'nz', 'v'} # Nouns, proper nouns, verbs

    words_with_pos = pseg.lcut(question_text) # List of (word, flag) pairs
    keywords = []
    seen_keywords = set()

    for word, flag in words_with_pos:
        word_clean = word.strip().lower()
        # Check if the word is not a stopword, not just whitespace,
        # and its POS tag (or the beginning of it, e.g., 'nrfg' starts with 'nr') is in our desired set.
        if word_clean and word_clean not in STOPWORDS and not word_clean.isspace():
            is_kept_pos = False
            for kept_tag in pos_tags_to_keep:
                if flag.startswith(kept_tag):
                    is_kept_pos = True
                    break
            
            if is_kept_pos:
                if word_clean not in seen_keywords:
                    keywords.append(word_clean)
                    seen_keywords.add(word_clean)
    return keywords

In [ ]:
我们做了词性分析来提高关键词提取的准确性，确保只提取有意义的词汇，这样BM25F会更好的考虑分数累计、命中数量。

准备评测的时候，我们发现代码评测速度有点慢，刚才我们写的代码里面有些地方值得优化，比如 query parser和search对象不应该反复创建。

![alt text](image-7.png)

这一次我们达到了还算满意的效果，0.8070左右


我在代码中还问了一个示例问题
"2014年南京青年奥林匹克运动会有哪些项目？"
这是从train.json随便找的一个问题。

![alt text](image-8.png)

结果发现系统检索结果为空，如果单独分别检索南京”、“奥林匹克运动会”和“哪些项目”都可以得到合理的答案。

仔细检查代码发现刚才评测到0.8070的代码没错，是我示例的代码忘记传入检索策略为OR了。

In [ ]:
qp = QueryParser("content", schema=index.schema, group=whoosh.qparser.OrGroup)

whoosh.qparser.OrGroup 是关键代码

## 2.构建问题分类器。

Huggingface Tranformers库非常适合做这一个子任务。



In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    pipeline
)


助教建议我们使用  DMetaSoul/sbert-chinese-general-v1 作为预训练模型。

我们可以使用 Hugging Face transformers 库和 Trainer API，结合最新的 Qwen 系列模型（我们将使用一个较小的 Qwen 版本，如 Qwen2-0.5B和Qwen-0.6B）来快速构建、微调和评估问题分类器。

同时 DMetaSoul/sbert-chinese-general-v1 也是很好加载的，接口是一样的。


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_fine_labels,
    id2label=id2label_fine_global,
    label2id=label2id_fine_global,
    trust_remote_code=MODEL_TRUST_REMOTE_CODE,
    ignore_mismatched_sizes=True # Important if resizing embeddings or changing classifier head
)

中途我们遇到了一些报错，比如pad token不存在

https://github.com/huggingface/transformers/blob/main/src/transformers/models/qwen3/modeling_qwen3.py

为了方便debug，我们定制了data collator

In [ ]:
class DebuggingDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        if 'labels' in batch:
            labels_tensor = batch['labels']
            print(f"DEBUG COLLATOR: Batch labels: min={labels_tensor.min().item() if labels_tensor.numel() > 0 else 'N/A'}, "
                  f"max={labels_tensor.max().item() if labels_tensor.numel() > 0 else 'N/A'}, "
                  f"shape={labels_tensor.shape}, num_fine_labels_config={num_fine_labels}")
            if labels_tensor.numel() > 0 and (labels_tensor.min() < 0 or labels_tensor.max() >= num_fine_labels):
                print(f"CRITICAL DEBUG COLLATOR: Label out of bounds! Min: {labels_tensor.min().item()}, Max: {labels_tensor.max().item()}. Expected range [0, {num_fine_labels-1}]")
        # You can also check input_ids here again if you suspect them despite earlier checks
        # if 'input_ids' in batch:
        #     input_ids_tensor = batch['input_ids']
        #     print(f"DEBUG COLLATOR: Batch input_ids: min={input_ids_tensor.min().item()}, max={input_ids_tensor.max().item()}")
        return batch

最后  DMetaSoul/sbert-chinese-general-v1 分类的效果如下
![alt text](image-10.png)
![alt text](image-9.png)

Fine-grained Accuracy: 0.8450
Coarse-grained Accuracy: 0.9289

而 Qwen3-0.6B 分类的效果如下

![alt text](image-12.png)
![alt text](image-11.png)


Fine-grained Accuracy: 0.9040
Coarse-grained Accuracy: 0.9471

以上训练使用了相同的 batch size，都是16，lr也是一样的 2e-5，训练NUM_EPOCHS = 3。

qwen3的效果明显更好，而且从示例问题的分类效果来看，输出的置信度高，qwen3更加自信自己的判断。看来这个大语言模型确实是很不错。

## 3. 抽取文档中最相关的候选答案句


我们首先学习一下评价指标MRR (Mean Reciprocal Ranking) 


/rɪˈsɪprək(ə)l/ 是 involving two people or groups who agree to help each other or behave in the same way to each other互惠的；相应的

比如 reciprocal arrangement 互惠协定

MRR非常简单，它衡量的是系统将 第一个相关项目 排在列表中的平均位置。MRR特别适用于那些用户通常**只关心找到第一个正确答案**的场景（例如，事实型问答系统、FAQ查找等）。

其实就是对于一个查询（Query），系统会返回一个排序后的项目列表，找到这个列表中第一个相关项目（Ground Truth）的排名（Rank），然后倒数。如果第一个就是那评分就是1，如果排在无穷后，就是0分。实际操作中模型不可能无限输出，MRR@K会截断，只搜索top k个结果没见多没检索到就是零分。

mean那就是对数据集所有query，平均这个值。

本次作业中只有一个ground truth，如果有多个groud truth怎么办呢？
MRR只管第一项，就是第一个出现的ground truth的排名，后面的ground truth当做没看见。


In [1]:
from torch import tensor
from torchmetrics.retrieval import RetrievalMRR
indexes = tensor([0, 0, 0, 1, 1, 1, 1])
preds = tensor([0.2, 0.3, 0.5, 0.1, 0.3, 0.5, 0.2])
target = tensor([False, False, True, False, True, False, True])
mrr = RetrievalMRR()
mrr(preds, target, indexes=indexes)


/home/ye_canming/micromamba/envs/ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor(0.7500)

在代码实现的过程中，我们遇到了一个关键的问题 https://github.com/scikit-learn/scikit-learn/issues/21335

也就是在没有负样本，只有正样本的情况下，NCDG指标无法计算。

虽然我们这次作业老师要求MRR，但是 Sentence Transformer https://sbert.net/docs/package_reference/cross_encoder/evaluation.html#crossencoderrerankingevaluator 捆绑实现了这两个指标。

于是我决定，修改 dev set，

![alt text](image-13.png)

对于只有没有负样本的情况，随机从其他问题中抽取一句话作为负样本。

In [ ]:
import random
print(dev_evaluator_data[0]) # Debug print to check the first item
length = len(dev_evaluator_data)
for i, item in enumerate(dev_evaluator_data):
    # 如果没有 negative，就从其他句子中随机抽取一些positive作为 negative
    negatives = item.get('negative', [])
    if len(negatives) == 0:
        # 从所有正面句子中随机抽取一些作为负面句子
        
        dev_evaluator_data[i]['negative'] = [
            dev_evaluator_data[i]['positive'][0]
            for i in random.sample(range(length), 3)
        ]

现在我们就成功使用了 https://sbert.net/docs/package_reference/cross_encoder/evaluation.html#crossencoderrerankingevaluator 库
的 from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator
来评测MRR@10指标。


接下来我们要构建模型

我参考 https://sbert.net/docs/quickstart.html#sentence-transformer 进行了学习。

了解到有两个东西，Sentence Transformer 是 向量检索，每一个Sentence变成vector算相似度。

但是一般 reranking 是用 cross encoder，输入的是一个pair，比较这两个pair的相似度，判断是相关还是不相关。

根据教程建议 https://sbert.net/examples/sentence_transformer/applications/retrieve_rerank/README.html 一般认为 cross encoder 效果更好，一般检索的项目只有100个以下的时候也不会太慢。

看到教程 https://sbert.net/docs/cross_encoder/training_overview.html 我突然恍然大悟，对于 cross encoder 而言，它也是能够完成刚才的任务二的，只需要比较句子和类型单词的相似度，就可以完成分类。



另外一个重要的问题是我们这里的问题和回答是不对称的，我们想计算问题和回答的相似度，而不是回答与回答的相似度。不是刚才第一个系统中做的文档与文档的相似度。

这个任务叫做 semantic search，是检索任务的一种。 https://sbert.net/docs/sentence_transformer/pretrained_models.html#multi-qa-models

从 Huggingface 的排行榜上，我可以看到一个比较新的 Text Ranking 任务的模型是 "Alibaba-NLP/gte-multilingual-reranker-base"

https://huggingface.co/Alibaba-NLP/gte-multilingual-reranker-base?library=sentence-transformers

人家是专门做这个semantic search的，效果应该不错，所以我们直接拿

In [ ]:
from sentence_transformers import CrossEncoder

model = CrossEncoder("Alibaba-NLP/gte-multilingual-reranker-base", trust_remote_code=True)

对于训练方式的选择

https://sbert.net/docs/cross_encoder/training_overview.html#loss-function

由于我决定使用 cross encoder 来做 reranking，而没有embedding model，所以没法做 hard mining，有时间可以再来优化。

按照教程的说法，虽然助教建议我们用对比学习的InfoNCE，但是对于 cross  encoder，binary cross entropy loss 是一个很难被超过的选择，所以我们直接使用 binary cross entropy loss 来训练 cross encoder。



经过大量代码的准备和调试，我们修复了很多个bug之后，终于成功运行

![alt text](image-14.png)

注意模型里面的坑有很多，比如Qwen3的pad token需要手动设置，rwkv需要进行适配。

labels是1还是2有一定的区别，一个用sigmoid，一个用softmax。

![alt text](image-15.png)

数据上也有很多坑，最后我们仔细阅读文档，最终处理好了数据，并且使用 Huggingface Dataset做了缓存。

训练一个epoch后得到结果

```python
2025-05-31 17:28:12 - MAP:     95.66
2025-05-31 17:28:12 - MRR@10:  95.77
2025-05-31 17:28:12 - NDCG@10: 96.69
```

![alt text](image-16.png)

我们想要比较融入上一个阶段问题类型分类的信息与否，是否能提高性能。

于是我设计了参数 USE_QUESTION_TYPE_FEATURE = False

在 = True的时候，数据处理后问题是

'[TYPE:OBJ] 《死亡日记(2009年上映美国电影) 》是一部什么类型的电影？'

= False 就是
'《死亡日记(2009年上映美国电影) 》是一部什么类型的电影？'

上面我们已经按照有这个信息的情况训练了一次，现在我们来看看没有这个信息的情况

```bash
2025-05-31 17:38:25 - MAP:     94.07
2025-05-31 17:38:25 - MRR@10:  94.15
2025-05-31 17:38:25 - NDCG@10: 95.50
```

可以看到，MAP、MRR和NDCG都下降了，说明问题类型信息确实有用，有助于检索提升性能。

## 4. 最相关的候选答案句中抽取最精简的答案

从我们上一阶段排序得到的最相关的候选答案句（或者直接使用train.json中的answer_sentence作为上下文进行训练）中，抽取出问题的具体答案。这个答案可能是一个词或者几个词（一个文本片段/span）。

根据“大作业指导书.docx”,这本质上是一个抽取式问答（Extractive Question Answering）或机器阅读理解（Machine Reading Comprehension, MRC）任务。给定一个问题 (Question) 和一个包含答案的上下文段落 (Context - 在我们的场景中，这就是最相关的候选答案句)，模型需要从上下文中找到答案所在的文本片段。

根据老师提示，可以转换为转换为序列标注问题 (Sequence Labeling)或者片段抽取式MRC使用的边界模型 (Boundary Model / Span Prediction Model)

上网查了下，Hugging Face transformers 库中的 AutoModelForQuestionAnswering 类就是为此类任务设计的，它通常在预训练模型（如BERT, Qwen2）的顶层添加了两个线性层来预测起始和结束位置的logits，所以第二种方法更加SOTA一点。

仔细学习了 https://huggingface.co/docs/transformers/tasks/question_answering 

SQuAD 是个重要的数据集。

形式大概是这样的

```json
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'
}
```


对于预训练模型，我决定采用 https://huggingface.co/timpal0l/mdeberta-v3-base-squad2


经过一系列调试，我们成功运行了代码

![alt text](image-17.png)

可以看到
1. 这个QA模型对于示例问题的回答还可以，正确的。
2. 对于acc指标，第一个epoch比第二个epoch高，说明模型在学习过程中有过拟合的情况。loss也是从0.54回升到了0.61
3. 'eval_start_token_acc': 0.7690839694656488, 'eval_end_token_acc': 0.7194656488549618

刚才训练过程中信息不足，难以评价复杂的指标（比如老师要求的BLEU），所以我又单独开了一个文件来对训练好保存下来的模型进行评测。

![alt text](image-18.png)

![alt text](image-19.png)

我们首先来理解一下这里计算指标的含义。

BLEU在上次英法机器翻译作业中是用过的，输出[0, 1]之间，这里0.14就是普通水平。


EM和F1使用了Hugging Face 的 evaluate.load("squad") ，内部封装了官方 SQuAD 评估逻辑的精确实现或非常接近的实现。它会处理分词、标准化和多答案比较。
单位不同， Exact Match (EM): 7.2519 应该是 7.25% 左右
F1 Score: 8.9332 同样，应该是 8.93% 左右

官方代码是 https://huggingface.co/spaces/evaluate-metric/squad




In [ ]:
import evaluate
metric = evaluate.load("evaluate-metric/squad")
metric?

Type:        Squad
String form:
EvaluationModule(name: "squad", module_type: "metric", features: {'predictions': {'id': Value(dty <...> ferences)
           >>> print(results)
           {'exact_match': 100.0, 'f1': 100.0}
           """, stored examples: 0)
Length:      0
File:        ~/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--squad/b4e2dbca455821c7367faa26712f378254b69040ebaab90b64bdeb465e4a304d/squad.py
Docstring:  
This metric wrap the official scoring script for version 1 of the Stanford Question Answering Dataset (SQuAD).

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by
crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span,
from the corresponding reading passage, or the question might be unanswerable.

Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
      


这个文档还是不清晰，于是我们硬刚源码

![alt text](image-20.png)


我们自己评测的F1定义和它稍有不同，我们的F1是自己使用jieba分词，计算出来的。我们的f1也是token level的，和Squad的区别在于可能没有进行复杂的文本标准化

In [6]:
from collections import Counter
from typing import List, Tuple
def compute_precision_recall_f1(prediction_tokens: List[str], ground_truth_tokens: List[str]) -> Tuple[float, float, float]:
    """
    Computes token-level precision, recall, and F1 score.
    """
    if not ground_truth_tokens: # If there's no ground truth, P/R/F1 are undefined or 0.
        return (1.0, 1.0, 1.0) if not prediction_tokens else (0.0, 0.0, 0.0)
    if not prediction_tokens: # If nothing is predicted, recall is 0. Precision is 1 if GT is also empty, else 0.
        return (1.0, 0.0, 0.0) if not ground_truth_tokens else (0.0, 0.0, 0.0)


    common_tokens = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_common = sum(common_tokens.values())

    precision = num_common / len(prediction_tokens) if len(prediction_tokens) > 0 else 0.0
    recall = num_common / len(ground_truth_tokens) if len(ground_truth_tokens) > 0 else 0.0
    
    f1 = 0.0
    if precision + recall > 0:
        f1 = (2 * precision * recall) / (precision + recall)
        
    return precision, recall, f1


不管怎么说，这个评测结果都告诉我们，性能很低。

性能很低，我决定另辟蹊径，不再使用question answering这个任务，而是使用text generation进行训练，快速进行SFT。因为从huggingface上来看，question ansewring的模型确实比较老旧，没有新的sota模型出来。
而且 QA 模型来做这个任务确实有局限性，刚才我们处理数据集的时候就已经发现，很多答案不是候选句子的子集，为此我们不得不丢弃了70多个数据。

为此，我首先参考了 https://huggingface.co/docs/trl/v0.18.1/en/sft_trainer#trl.SFTConfig 这个 参数还是挺复杂的。

然后我仔细阅读了 https://modelscope.cn/notebook/share/ipynb/d4d8765f/qwen3.ipynb
这里面对于 “深度思考” 的训练和推理方法介绍的比较好，对于qwen3这个模型，训练集我们没有思维链，那我们就把enable_thinking设置为False
但是评测的时候允许模型思考，来增强它的能力。

经过大量的调试和尝试，我们终于成功运行了代码

![alt text](image-21.png)


这个qwen3 0.6 B 的 SFT 微调还是挺占显存的，使用了 32G， BATCH_SIZE_SFT = 4

![alt text](image-22.png)


最后我们评测

![alt text](image-23.png)

f1 和 rouge-l 的值比较合理，EM和BLEU 似乎评测的有点不对。可能是我们去除思维链进行评测的函数没写对，但是代码太复杂了，暂时没有debug成功。

## 整体流程

现在我们终于完成了前面的四步，可以形成一个整体pipeline为一个完整问答系统了。

我写一个 overall.py 来整合前面的四个部分。

然而代码也很复杂，特别是 whoosh代码很不优雅，需要全部复制一遍过来。


不过还好我们还是结合AI一起仔细检查了代码，确保推理流程完整正确。

现在我们终于跑通了整个流程：文档检索、问题分类、候选答案句排序和答案抽取。

如图所示，我们的系统还是很厉害的，可以完整地检索文档、分析问题类型、排序句子、提取答案。
![alt text](image-24.png)


最终对测试集上评测，结果如下

![alt text](image-25.png)

exact match 达到了 28%, F1 score 达到了 34.17%

同时也要注意到failure case也很多

比如whoosh文档检索直接失败的情况很多

![alt text](image-26.png)


大模型提取答案提取失败的情况也存在。
![alt text](image-27.png)

这里对于评测指标有一个值得注意的问题，BLEU特别低！
其实我思考了一下，还是可以理解的，
BLEU 最初是为机器翻译设计的，它通过比较候选文本和参考文本之间n-gram（通常是1到4-gram）的重叠来工作。

这里我们回答的答案一般就一两个词！高阶n-gram的精度会非常低，导致整体BLEU分数急剧下降，甚至为0。

此外，evaluate 库中的 bleu 指标在处理中文时，其内部的分词方式非常重要。如果它默认按字符进行n-gram计算（character-level BLEU），那么对于基于词的匹配会非常严格。例如，“北京市”和“北京”在字符级别几乎没有2-gram以上的重叠。

我们rouge是特意为中文选择了 rouge chinese 库，但是bleu我没找到，所以没有仔细搞，所以这个指标算出来不准确。


## 附加题：使用提高文档检索的能力

前面我们第一题用的是 term-based 的检索方法，使用了 Whoosh 库，实际上是BM25F 算法。
第三题使用了 cross encoder 来对候选答案句进行排序，实际上是semantic based 的检索方法。

助教和老师非常厉害，视野很前沿，在附加实验中给我们展示了一个新的思路，基于A Neural Corpus Indexer for Document Retrieval，NIPS 2022 文章。

引入了语义标识符（semantic identifiers）的概念。看了助教的介绍，我感觉首先这个semantic 还是需要文档向量，计算相似度，我们需要选一个 Sentence Transformer 模型来生成文档向量。然后这个kmeans 层次化聚类就比较厉害了，会有一个超参数c，树状的分很多层类。

我决定使用  https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 作为 Sentence Transformer，为文档生成向量，这个模型比BERT之类的新，应该性能更好。

然后设置 C_PARAM = 10 每一层有 10个类，层次化聚类。

![alt text](image-28.png)

成功生成

为了避免重复生成，我们把结果保存到 semantic_identifiers.json 文件中。

![alt text](image-29.png)

现在我们再次使用Qwen3，使用SFT来训练，强行要求模型生成这个语义标签的字符串。
这有什么好处呢？按照层次化的思想，由于生成字符串从左到右，所以先生成了粗类，后生成了细类。

上面的任务我们已经深入了解了 SFT 要怎么训练，现在我们再次把代码拿下来。

经过对SFT Trainer 参数的深度理解，以及我们对数据进行了合适的处理，终于跑通了

In [ ]:
def format_sft_semid_example(question: str, semantic_id_target: str, tokenizer: AutoTokenizer) -> Optional[str]:
    """
    Formats a single example for SFT to generate semantic identifiers.
    The model learns to predict the semantic_id_target.
    """
    system_message = "你是一个AI助手，你的任务是根据用户提出的问题，为其推荐最相关文档的语义标识符。语义标识符是一串由连字符'-'分隔的数字。"
    user_message_content = f"问题：\n{question}\n\n请为上述问题生成最相关文档的语义标识符。"
    
    # The target completion is the semantic ID string
    assistant_message_content = semantic_id_target

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message_content},
        {"role": "assistant", "content": assistant_message_content}
    ]
    
    try:
        formatted_text = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=False # False, as SFTTrainer expects the full text including completion
        )
        # SFTTrainer usually expects the text to end with EOS if it's not added by template
        # if not formatted_text.endswith(tokenizer.eos_token):
        #    formatted_text += tokenizer.eos_token
        return formatted_text
    except Exception as e:
        logger.warning(f"SFT SemID: Error applying chat template for {tokenizer.name_or_path}: {e}. Using generic fallback.")
        prompt_part = f"<s>[INST] 系统: {system_message}\n用户: {user_message_content} [/INST]\n"
        completion_part = assistant_message_content
        return prompt_part + completion_part + tokenizer.eos_token

![alt text](image-30.png)